In [ ]:
#Library importation
import pandas as pd

In [ ]:
#loading data

df = pd.read_csv("insurance_data_governance_simulation.csv")

In [ ]:
#viewing data
df.head()

,customer_id,full_name,date_of_birth,policy_number,policy_type,premium_amount,claim_amount,claim_status,registration_date
0,1038,Client_0,1998-01-07,5014,General,NaN,18323,NaN,2024-04-11 00:00:00
1,1028,Client_1,1988-03-17,5026,Life,61576.0,39121,approved,2022-01-22 00:00:00
2,1014,Client_2,1993-12-10,5021,Life,40358.0,15975,Rejected,2020-05-09 00:00:00
3,1042,Client_3,1999-07-27,5023,Life,23267.0,16023,rejected,2024-11-14 00:00:00
4,1007,Client_4,1998-05-20,5008,Life,37209.0,26447,Approved,2020-11-02 00:00:00


In [ ]:
df.tail()

,customer_id,full_name,date_of_birth,policy_number,policy_type,premium_amount,claim_amount,claim_status,registration_date
200,1007,Client_192,1992-05-07,5016,Life,62356.0,22302,APPROVED,2020-03-04 00:00:00
201,1036,Client_157,1992-12-31,5002,General,59298.0,44144,APPROVED,2024-12-06 00:00:00
202,1007,Client_108,1972-03-03,5008,Life,43322.0,15209,Rejected,2024-11-08 00:00:00
203,1005,Client_173,1995-01-08,5021,Life,42700.0,34721,APPROVED,2024-02-18 00:00:00
204,1029,Client_156,1984-04-26,5006,Medical,72224.0,44749,Rejected,2024-04-19 00:00:00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        205 non-null    int64 
 1   full_name          205 non-null    object
 2   date_of_birth      205 non-null    object
 3   policy_number      205 non-null    int64 
 4   policy_type        205 non-null    object
 5   premium_amount     194 non-null    object
 6   claim_amount       205 non-null    int64 
 7   claim_status       196 non-null    object
 8   registration_date  205 non-null    object
dtypes: int64(3), object(6)
memory usage: 14.5+ KB


Data Quality Assessment

In [ ]:
#checking for duplicates
df.duplicated().sum()

np.int64(5)

In [ ]:
#missing values
df.isnull().sum()

,0
customer_id,0
full_name,0
date_of_birth,0
policy_number,0
policy_type,0
premium_amount,11
claim_amount,0
claim_status,9
registration_date,0


In [ ]:
#checking for inconsistencies
df['claim_status'].value_counts()

,count
claim_status,
Rejected,42
APPROVED,41
approved,40
rejected,40
Approved,33


###Root Cause Analysis
From the data, the following were observed:
#Duplicate policy numbers
- There are 5 duplicates
- Same policy number appears twice
- This implies that the system allows manual entry without validation
- The solution is to enforce uniqueness, add validation rules

#Missing premium amount
- There are 11 records blank
- Employees skip entry or system error
- to solve this,mandatory field and automated data checks should be implemented.

#Inconsistent claim status
- there include: “approved”, “Approved”, “APPROVED”
- This shows no controlled vocabulary
- It can be mitigated by standardizing values and enforcing allowed values

###Data Cleaning

In [ ]:
#removing duplicates
df.drop_duplicates()

,customer_id,full_name,date_of_birth,policy_number,policy_type,premium_amount,claim_amount,claim_status,registration_date
0,1038,Client_0,1998-01-07,5014,General,NaN,18323,NaN,2024-04-11 00:00:00
1,1028,Client_1,1988-03-17,5026,Life,61576.0,39121,approved,2022-01-22 00:00:00
2,1014,Client_2,1993-12-10,5021,Life,40358.0,15975,Rejected,2020-05-09 00:00:00
3,1042,Client_3,1999-07-27,5023,Life,23267.0,16023,rejected,2024-11-14 00:00:00
4,1007,Client_4,1998-05-20,5008,Life,37209.0,26447,Approved,2020-11-02 00:00:00
...,...,...,...,...,...,...,...,...,...
195,1036,Client_195,1990-03-28,5029,Medical,23330.0,6531,rejected,2024-03-15 00:00:00
196,1032,Client_196,1994-10-16,5028,Medical,39087.0,45940,rejected,2021-08-13 00:00:00
197,1041,Client_197,1980-04-09,5013,Medical,61430.0,45441,Rejected,2020-10-10 00:00:00
198,1043,Client_198,1969-06-21,5020,Medical,44504.0,28333,Rejected,2024-11-13 00:00:00


In [ ]:
#handling missing values
df["claim_status"].fillna("Pending Review", inplace=True)

/tmp/ipython-input-3931980594.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["claim_status"].fillna("Pending Review", inplace=True)


In [ ]:
#ensuring all values in the premium_amount column are numeric
df["premium_amount"] = pd.to_numeric(df["premium_amount"], errors="coerce")

In [ ]:
df["premium_missing_flag"] = df["premium_amount"].isnull().astype(int)

In [ ]:
median_premium = df["premium_amount"].median()
df["premium_amount"].fillna(median_premium, inplace=True)

/tmp/ipython-input-2199931221.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["premium_amount"].fillna(median_premium, inplace=True)


In [ ]:
#handling inconsistencies
df['claim_status'] = df['claim_status'].replace({'rejected':'Rejected','APPROVED':'Approved','approved':'Approved'})

##Data cleaning
- dropped all duplicated rows to ensure uniqueness in data
- the missing entries claim_status column filling with a constant( pending review)
- converted all records in the preminum_amount column to numeric values and used coerce to convert unknowm values to NaN.
- filled missing values in preminum_amount column with the median.
- Added a flagged column to show originally missing values. This preserves the audit trail

##Cleaning confirmation

In [ ]:
#missing values
df.isnull().sum()

,0
customer_id,0
full_name,0
date_of_birth,0
policy_number,0
policy_type,0
premium_amount,0
claim_amount,0
claim_status,0
registration_date,0
premium_missing_flag,0


In [ ]:
#consistency of data
df['claim_status'].value_counts()

,count
claim_status,
Approved,114
Rejected,82
Pending Review,9
